# Extract chemgauss results

This notebook takes `data_optim.csv` (outputted by 5_extract_geometry_stabilities). This notebook then creates the new csv `data_all.csv` which contains all the results from this project neccessary to create the metrics figure.

In [1]:
import pandas as pd  
import glob

In [41]:
# SILVR + EDM, fragments 0072 + 0107
# experiment_names = ['exp_9','exp_10','exp_11','exp_12','exp_13','exp_14','exp_15']

# BRIDGE + EDM, fragments 0072 + 0107
# experiment_names = ['exp_16','exp_17','exp_18','exp_19','exp_20','exp_21','exp_22']
# BRIDGE + EDM, fragments 0072 + 0107 (250 SAMPLES)
# experiment_names = ['exp_23','exp_24','exp_25','exp_26','exp_27','exp_28','exp_29']

# BRIDGE + EDM, fragments 0072 + 0107 (Explicit Hydrogens)
experiment_names = ['exp_30','exp_31','exp_32','exp_33','exp_34','exp_35','exp_36']
#TODO: Fix inconsistent naming of variables
exp_names = experiment_names
oe_exp_list = [f"od_docking_data/{x}/" for x in exp_names]#oe without water
# oe_exp_list = [f"od_docking_data/{x}/" for x in exp_names]# TODO: oe with water
experiment_path = "experiments/"

In [42]:
def oe_remove_duplicate_data(oe_df):
    ''' This function takes in a dataframe oe_df, sorts it based on specific columns, removes duplicate rows, concatenates the unique rows with NaN rows, sorts the final dataframe, and returns the cleaned dataframe.'''

    # First, sort the dataframe by 'ligand_name', 'protein_name', and 'computed_dG' to ensure the lowest values come first
    # if ligand_name is present, sort by ligand_name, protein_name, and computed_dG
    if {'ligand_name', 'protein_name'}.issubset(oe_df.columns):
        try:
            df_sorted = oe_df.sort_values(by=['ligand_name', 'protein_name', 'computed_dG'])
        except:
            df_sorted = oe_df.sort_values(by=['ligand_name', 'protein_name', 'Chemgauss4'])
    else:
        # if ligand_name is not present, sort by pdb_id and computed_dG
        df_sorted = oe_df.sort_values(by=['pdb_id', 'computed_dG'])

    # Separate the rows where 'computed_dG' is NaN
    nan_rows = df_sorted[df_sorted['computed_dG'].isna()]

    # Drop NaN rows from the sorted dataframe to process only valid dG values
    valid_rows = df_sorted.dropna(subset=['computed_dG'])

    # Drop duplicates, keeping the first occurrence which has the lowest 'computed_dG' due to sorting
    if {'ligand_name', 'protein_name'}.issubset(oe_df.columns):
        unique_rows = valid_rows.drop_duplicates(subset=['ligand_name', 'protein_name'], keep='first')
    else: 
        unique_rows = valid_rows.drop_duplicates(subset=['pdb_id'], keep='first')

    # Concatenate the unique rows with the NaN rows
    final_df = pd.concat([unique_rows, nan_rows])

    # Optionally, sort final dataframe by index or any other column if needed
    final_df = final_df.sort_index()

    # Save or display the final dataframe
    # final_df.to_csv('leakypdb_test_OE_w_water_data_cleaned_unique_rows.csv', index=False)
    return final_df

In [43]:
# oe scores are already written to a csv file so there's no need to extract even further.
oe_df_list = [f"{x}_mpro_oe_docking_data.csv" for x in exp_names]
oe_results_df = pd.DataFrame()
# read in all the df from oe_df_list
for path in oe_df_list:
    oe_df = pd.read_csv(path)
        # get rid of the first row if the first column is not 'ligand_name'
    if oe_df.columns[0] != 'ligand_name':
        oe_df = oe_df.iloc[1:]
        #drop the error_message column
        oe_df = oe_df.drop(columns=['error_message'])
        #rename the columns
        oe_df = oe_df.rename(columns={'computed_dG':'error_message','protein_name':'computed_dG','ligand_name':'protein_name','Unnamed: 0':'ligand_name'})
    oe_df = oe_remove_duplicate_data(oe_df)
    oe_df_subset = oe_df[['ligand_name', 'computed_dG']]
    df = pd.DataFrame(oe_df_subset)
    df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
    # get rid of the _0 in the ligand_name column
    df['ligand_name'] = df['ligand_name'].str.replace(r'(_0)$', '',regex=True)
    oe_results_df = pd.concat([oe_results_df, df])
    
oe_results_df = oe_results_df.rename(columns={'ligand_name': 'mol_id', 'computed_dG': 'chemgauss4'})
oe_results_df

/tmp/ipykernel_3530417/642919417.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
/tmp/ipykernel_3530417/642919417.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
/tmp/ipykernel_3530417/642919417.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
/tmp/ipykernel_3530417/642919417.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
/tmp/ipy

,mol_id,chemgauss4
0,mol_2024_07_10_1800131_000,-6.241024
2,mol_2024_07_10_1754523_000,NaN
3,mol_2024_07_10_1749093_000,NaN
4,mol_2024_07_10_1757000_000,-7.335155
6,mol_2024_07_10_1745590_000,NaN
...,...,...
99,mol_2024_07_10_2047213_000,NaN
100,mol_2024_07_10_2044152_000,NaN
101,mol_2024_07_10_2040111_000,NaN
102,mol_2024_07_10_2039161_000,NaN


In [44]:
old_df = pd.read_csv(experiment_path+f"{experiment_names[0]}_to_{experiment_names[-1]}_data_all.csv")
df = pd.merge(old_df,oe_results_df, on="mol_id",how='left')
df.to_csv(experiment_path+f"{experiment_names[0]}_to_{experiment_names[-1]}_data_all.csv", index=False)
df

,mol_id,total_atoms,stable_ratio,exp_id,dummy_atoms,samples,silvr,comment,crude_rmsd,is_fragmented,qed,sa,smiles,auto3d_rmsd,vina_score,chemgauss4
0,mol_2024_07_10_1802383_000,24,0.791667,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.176292,False,0.748112,6.850152,[H]c1sc2c(c1[H])=NN([H])c1nc(C([H])([H])[H])c(...,0.659048,-6.502,-6.593245
1,mol_2024_07_10_1746031_000,24,0.750000,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.468629,False,0.115395,4.058768,[H]N(N=[N+]=[N-])NC([H])(NNC([H])(OC(=O)C([H])...,1.751093,-5.741,-6.284486
2,mol_2024_07_10_1751470_000,24,0.708333,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.623005,True,0.648801,NaN,[H]C([H])([H])N(C([H])([H])C([H])([H])[H])S(O)...,NaN,NaN,NaN
3,mol_2024_07_10_1802540_000,24,0.916667,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.186990,False,0.467546,7.333780,[H]Oc1c([H])c(N([H])[H])c([H])c(=O)c(N([H])C([...,0.874503,-5.062,-5.712659
4,mol_2024_07_10_1806243_000,24,0.708333,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.255156,False,0.145665,7.434265,[H]N(C(=O)N1C([H])(C([H])([H])NN([H])C([H])([H...,1.124797,-4.848,-5.981421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,mol_2024_07_10_2047480_000,47,0.000000,exp_36,0,100,0.03,(BRIDGE + EDM) Effect of SILVR rate on samplin...,4.172006,True,0.212535,NaN,F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.FF.FF.FF.FF.FF.F...,NaN,NaN,NaN
724,mol_2024_07_10_2053181_000,47,0.000000,exp_36,0,100,0.03,(BRIDGE + EDM) Effect of SILVR rate on samplin...,4.139319,True,0.212511,NaN,F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.FF.FF.FF...,NaN,NaN,NaN
725,mol_2024_07_10_2031292_000,47,0.000000,exp_36,0,100,0.03,(BRIDGE + EDM) Effect of SILVR rate on samplin...,4.146521,True,0.212547,NaN,F.F.F.F.F.F.F.F.F.F.F.F.F.FF.FF.FF.FF.FF.FF.FF...,NaN,NaN,NaN
726,mol_2024_07_10_2039161_000,47,0.000000,exp_36,0,100,0.03,(BRIDGE + EDM) Effect of SILVR rate on samplin...,4.201126,True,0.212489,NaN,F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F....,NaN,NaN,NaN
